In [1]:
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format

In [2]:
def linear_congruential_generator(N, seed=1):
    """Generates uniform random samples on [0,1]."""
    
    ## Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = seed

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui
    
    return samples, xi  # return the last generated value, xi, as the updated seed

## Test Script:
# N = 10
# print(linear_congruential_generator(N))

In [3]:
def inverse_normal_approximation(u):
    """Generates normally distributed realizations from uniform random realizations."""
    
    ## Constants for approximations to inverse normal.
    a0, a1, a2, a3 = 2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637
    b0, b1, b2, b3 = -8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833
    c0, c1, c2, c3 = 0.3374754822726147, 0.9761690190917186, 0.1607979714918209, 0.0276438810333863
    c4, c5, c6, c7, c8 = 0.0038405729373609, 0.0003951896511919, 0.0000321767881768, 0.0000002888167364, 0.0000003960315187

    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0:
            r = 1 - u
        r = math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4 + r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0:
            x = -x

    return x

## Test Script:
# u = 0.8
# print(inverse_normal_approximation(u))
## Results verified here: https://statisticshelper.com/inverse-normal-distribution-calculator/#answer

## Test Script 2:
# N = 1000
# k = 5
# z = np.vectorize(inverse_normal_approximation)(linear_congruential_generator(N*(2**k)))
# plt.hist(z)

In [4]:
def box_muller(u1,u2):
    u1 = 2 * u1 - 1
    u2 = 2 * u2 - 1
    x = u1**2 + u2**2
    if x > 1:
        return None
    y = np.sqrt(-2*math.log(x)/x)
    z1 = u1 * y
    z2 = u2 * y
    return (z1,z2)

# Variance Reduction Techniques for Monte Carlo Pricing of European Options

In [5]:
def marsaglia_bray(seed=1):
    X = 2  # set an initial value so that the while loop is entered
    while X > 1:
        u, seed = linear_congruential_generator(2, seed)  # get updated seed
        u1, u2 = u[0], u[1]
        
        u1 = 2 * u1 - 1
        u2 = 2 * u2 - 1
        X = u1**2 + u2**2

    Y = math.sqrt(-2 * math.log(X) / X)
    Z1 = u1 * Y
    Z2 = u2 * Y
    
    return Z1, Z2, seed  # return the updated seed value

## Test Script:
# print(marsaglia_bray())

In [6]:
def marsaglia_bray_N(N):

    ## Generate an empty array for the generated values.
    results = np.empty(N, dtype=float)
    index = 0
    seed = 1

    while index < N:
        Z1, Z2, seed = marsaglia_bray(seed)
        
        ## Add the generated values to the results array.
        results[index] = Z1
        index += 1
        
        ## If there's still space, add the second generated value.
        if index < N:
            results[index] = Z2
            index += 1

    return results

## Test Script:
# N = 10_000
# plt.hist(marsaglia_bray_N(N), bins=100)

In [7]:
def simulated_spot(S0, T, r, q, sigma, zi):
    """Calculate the simulated spot price S_i at time T."""
    return S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * zi)

def put_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European put option at time T."""
    return np.exp(-r * T) * np.maximum(K - Si, 0)

In [8]:
def d1(S, K, T, r, q, sigma):
    """Calculates d1 (BSM)."""
    return (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S, K, T, r, q, sigma, d1_val=None):
    """Calculates d2 (BSM)."""
    if d1_val is None:
        d1_val = d1(S, K, T, r, q, sigma)
    return d1_val - sigma * math.sqrt(T)

def bs_call(S, K, T, r, q, sigma):
    """Calculate the value for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return S * math.exp(-q * T) * norm.cdf(d1_val) - K * math.exp(-r * T) * norm.cdf(d2_val)

def bs_put(S, K, T, r, q, sigma):
    """Calculate the value for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return K * math.exp(-r * T) * norm.cdf(-d2_val) - S * math.exp(-q * T) * norm.cdf(-d1_val)

def bs_call_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return math.exp(-q * T) * norm.cdf(d1_val)

def bs_put_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return -math.exp(-q * T) * norm.cdf(-d1_val)

def bs_gamma(S, K, T, r, q, sigma):
    """Calculate the gamma of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return np.exp(-q * T) * norm.pdf(d1_val) / (S * sigma * math.sqrt(T))

def bs_vega(S, K, T, r, q, sigma):
    """Calculate the vega of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return S *  math.exp(-q * T) * math.sqrt(T) * norm.pdf(d1_val)

## Test Script:
# S = 100
# K = 100
# T = 1
# r = 0.03
# q = 0.03
# sigma = 0.2

# print("Call Option Value:", bs_call(S, K, T, r, q, sigma))
# print("Put Option Value:", bs_put(S, K, T, r, q, sigma))
# print("Call Option Delta:", bs_call_delta(S, K, T, r, q, sigma))
# print("Put Option Delta:", bs_put_delta(S, K, T, r, q, sigma))
# print("Option Gamma:", bs_gamma(S, K, T, r, q, sigma))
# print("Option Vega:", bs_vega(S, K, T, r, q, sigma))
## Verified results are correct here: https://goodcalculators.com/black-scholes-calculator/

In [9]:
def HW2_NUM1_CV(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)
    ##plt.hist(Si_array, bins=10)
    
    ## Vectorized calculation for put payoff.
    Vi_array = put_payoff(Si_array, K, T, r)
    ##plt.hist(Vi_array, bins=10)

    ## Aggregation.
    S_hat = np.mean(Si_array)
    V_hat = np.mean(Vi_array)

    ## Control Variate Technique.
    b_hat = np.sum((Si_array - S_hat) * (Vi_array - V_hat)) / np.sum((Si_array - S_hat)**2)
    Wi_array = Vi_array - b_hat * (Si_array - np.exp(r * T) * S0)
    V_cv_hat = np.mean(Wi_array)

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_CV_hat(N):", V_cv_hat)
    #print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_CV_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_cv_hat))
    print("\n")

In [10]:
## Table values for control variate technique.

## Given values.
S0 = 56
K = 54
sigma = 0.27
q = 0
r = 0.02
T = 0.75
N = 10_000 * (2**9)

z = marsaglia_bray_N(N)

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_CV(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_CV_hat(N): 3.762976882118533
|V_BS - V_CV_hat(N)|: 0.038095001708303045


Results for N = 20000
V_CV_hat(N): 3.760174727915477
|V_BS - V_CV_hat(N)|: 0.04089715591135912


Results for N = 40000
V_CV_hat(N): 3.798584417438943
|V_BS - V_CV_hat(N)|: 0.0024874663878931003


Results for N = 80000
V_CV_hat(N): 3.805737820114191
|V_BS - V_CV_hat(N)|: 0.004665936287354899


Results for N = 160000
V_CV_hat(N): 3.811181961648319
|V_BS - V_CV_hat(N)|: 0.010110077821483099


Results for N = 320000
V_CV_hat(N): 3.805990838805181
|V_BS - V_CV_hat(N)|: 0.004918954978344825


Results for N = 640000
V_CV_hat(N): 3.804925924377765
|V_BS - V_CV_hat(N)|: 0.003854040550928861


Results for N = 1280000
V_CV_hat(N): 3.8029039866601155
|V_BS - V_CV_hat(N)|: 0.001832102833279503


Results for N = 2560000
V_CV_hat(N): 3.801071882788615
|V_BS - V_CV_hat(N)|: 1.0382210646753265e-09


Results for N = 5120000
V_CV_hat(N): 3.800667107236537
|V_BS - V_CV_hat(N)|: 0.0004047765902992495




In [11]:
def HW2_NUM1_AV(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si1_array = simulated_spot(S0, T, r, q, sigma, z)
    Si2_array = simulated_spot(S0, T, r, q, sigma, -z)
    
    ## Antithetic Variables Technique.
    Vi1_array = put_payoff(Si1_array, K, T, r)
    Vi2_array = put_payoff(Si2_array, K, T, r)
    V_av_hat = (np.mean(Vi1_array) + np.mean(Vi2_array)) / 2

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_AV_hat(N):", V_av_hat)
    #print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_AV_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_av_hat))
    print("\n")

In [12]:
## Table values for anthithetic variables technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_AV(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_AV_hat(N): 3.765569398350365
|V_BS - V_AV_hat(N)|: 0.03550248547647117


Results for N = 20000
V_AV_hat(N): 3.765214430303043
|V_BS - V_AV_hat(N)|: 0.035857453523793215


Results for N = 40000
V_AV_hat(N): 3.7970454621203027
|V_BS - V_AV_hat(N)|: 0.004026421706533334


Results for N = 80000
V_AV_hat(N): 3.8034121552159017
|V_BS - V_AV_hat(N)|: 0.0023402713890656734


Results for N = 160000
V_AV_hat(N): 3.80809285928284
|V_BS - V_AV_hat(N)|: 0.007020975456003775


Results for N = 320000
V_AV_hat(N): 3.8051454410796604
|V_BS - V_AV_hat(N)|: 0.0040735572528243225


Results for N = 640000
V_AV_hat(N): 3.8047689469131014
|V_BS - V_AV_hat(N)|: 0.003697063086265384


Results for N = 1280000
V_AV_hat(N): 3.802765668873329
|V_BS - V_AV_hat(N)|: 0.0016937850464930904


Results for N = 2560000
V_AV_hat(N): 3.800853973646405
|V_BS - V_AV_hat(N)|: 0.00021791018043115784


Results for N = 5120000
V_AV_hat(N): 3.800499608915894
|V_BS - V_AV_hat(N)|: 0.0005722749109420278




In [13]:
def HW2_NUM1_MM(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)

    ## Aggregation.
    S_hat = np.mean(Si_array)

    ## Moment matching technique.
    Si_tilda_array = (np.exp(r*T)*S0/S_hat) * Si_array
    Vi_tilda_array = put_payoff(Si_tilda_array, K, T, r)
    V_mm_hat = np.mean(Vi_tilda_array)

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_MM_hat(N):", V_mm_hat)
    #print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_MM_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_mm_hat))
    print("\n")

In [14]:
## Table values for moment matching technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_MM(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_MM_hat(N): 3.758879248746038
|V_BS - V_MM_hat(N)|: 0.04219263508079818


Results for N = 20000
V_MM_hat(N): 3.7584481485562944
|V_BS - V_MM_hat(N)|: 0.04262373527054164


Results for N = 40000
V_MM_hat(N): 3.797759051858886
|V_BS - V_MM_hat(N)|: 0.0033128319679498475


Results for N = 80000
V_MM_hat(N): 3.805040627815494
|V_BS - V_MM_hat(N)|: 0.0039687439886577636


Results for N = 160000
V_MM_hat(N): 3.8100138105616193
|V_BS - V_MM_hat(N)|: 0.008941926734783223


Results for N = 320000
V_MM_hat(N): 3.806129769131595
|V_BS - V_MM_hat(N)|: 0.0050578853047591465


Results for N = 640000
V_MM_hat(N): 3.8053177682133223
|V_BS - V_MM_hat(N)|: 0.0042458843864863205


Results for N = 1280000
V_MM_hat(N): 3.8030411835162137
|V_BS - V_MM_hat(N)|: 0.001969299689377646


Results for N = 2560000
V_MM_hat(N): 3.8009817806465915
|V_BS - V_MM_hat(N)|: 9.010318024449404e-05


Results for N = 5120000
V_MM_hat(N): 3.800750041218082
|V_BS - V_MM_hat(N)|: 0.00032184260875389725




In [15]:
def HW2_NUM1_CVMM(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)

    ## Aggregation.
    S_hat = np.mean(Si_array)

    ## Moment matching technique.
    Si_tilda_array = (np.exp(r*T)*S0/S_hat) * Si_array
    Vi_tilda_array = put_payoff(Si_tilda_array, K, T, r)
    V_mm_hat = np.mean(Vi_tilda_array)

    ## Control Variate Technique.
    b_hat = np.sum((Si_tilda_array - np.exp(r*T)*S0) * (Vi_tilda_array - V_mm_hat)) / np.sum((Si_tilda_array - np.exp(r*T)*S0)**2)
    Wi_array = Vi_tilda_array - b_hat * (Si_tilda_array - np.exp(r * T) * S0)
    V_cvmm_hat = np.mean(Wi_array)
    
    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_CV,MM_hat(N):", V_mm_hat)
    #print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_CV,MM_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_cvmm_hat))
    print("\n")

In [16]:
## Table values for simultaneous moment matching and control variates technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_CVMM(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_CV,MM_hat(N): 3.758879248746038
|V_BS - V_CV,MM_hat(N)|: 0.04219263508080173


Results for N = 20000
V_CV,MM_hat(N): 3.7584481485562944
|V_BS - V_CV,MM_hat(N)|: 0.04262373527054297


Results for N = 40000
V_CV,MM_hat(N): 3.797759051858886
|V_BS - V_CV,MM_hat(N)|: 0.003312831967947627


Results for N = 80000
V_CV,MM_hat(N): 3.805040627815494
|V_BS - V_CV,MM_hat(N)|: 0.003968743988655987


Results for N = 160000
V_CV,MM_hat(N): 3.8100138105616193
|V_BS - V_CV,MM_hat(N)|: 0.008941926734785888


Results for N = 320000
V_CV,MM_hat(N): 3.806129769131595
|V_BS - V_CV,MM_hat(N)|: 0.005057885304765808


Results for N = 640000
V_CV,MM_hat(N): 3.8053177682133223
|V_BS - V_CV,MM_hat(N)|: 0.004245884386488541


Results for N = 1280000
V_CV,MM_hat(N): 3.8030411835162137
|V_BS - V_CV,MM_hat(N)|: 0.001969299689383419


Results for N = 2560000
V_CV,MM_hat(N): 3.8009817806465915
|V_BS - V_CV,MM_hat(N)|: 9.01031802422736e-05


Results for N = 5120000
V_CV,MM_hat(N): 3.800750041218

# Monte Carlo Pricing for Basket Options

In [17]:
T = 0.5
r = 0.025
S10 = 26
S20 = 29
v1 = 0.31
v2 = 0.21
K = 50
rho = 0.3

In [18]:
#Monte Carlo Pricing for Basket Options
k = list(range(9))
for _k in k:
    n = 10000*(2**_k)
    normalSamples = marsaglia_bray_N(2*n)
    
    Z1 = np.array(normalSamples)[::2]
    Z2 = np.array(normalSamples)[1::2]
    
    Zero = np.zeros(len(Z1)) 
    
    S1 = S10*np.exp(((r-((v1**2)/2))*T) + (v1*(math.sqrt(T))*Z1))
    S2 = S20*np.exp(((r-((v2**2)/2))*T) + (v2*(math.sqrt(T))*(rho*Z1 + math.sqrt(1-(rho**2))*Z2)))
    
    V = math.exp(-r*T)*np.maximum(S1 + S2 - K , Zero)
    
    V_n = np.mean(V)
    
    print( V_n)

6.635690069726578
6.654352366186453
6.640069192179972
6.641466472571224
6.662836155284208
6.6597729706860775
6.6548515760518345
6.649923570630976
6.652679426499821


# Monte Carlo Pricing for Path–Dependent Basket Options

In [19]:
def payoff_lockback_basket_call_option(S_1, S_2, K):
    return np.max(S_1+S_2-K, 0)

In [20]:
def generate_normal_samples(uniSamples):
    samples = []
    N = 10000*(2**9)
    i = 0
    while len(samples) < N:
        Z = box_muller(uniSamples[i] , uniSamples[i+1])
        if Z:
            samples.append(Z[0])
            samples.append(Z[1])
        i += 2
    return samples

In [21]:
# parameter of the asset
S_1_0 = 26
S_2_0 = 29
sigma_1 = 0.31
sigma_2 = 0.21
rho = 0.3
r = 0.025
K = 50

T = 0.5

In [22]:
# page 6
df_result_page_6 = pd.DataFrame(columns = ['N_k' , 'm=150' , 'n' , 'V^(n)'])
m = 150 # fixed discretization
k = list(range(10))
delta_t = T / m

for _k in k:
    sum_V = 0
    n = 50*(2**_k) # paths
    N_k = 2 * m * n # 2 * 150 * 50 * 2**k # number of normal rvs required
    U, _ = linear_congruential_generator(int(3 * N_k))
    count = 0
    i = 0
    while count < 2 * m * n:
        time = 0
        S_1 = S_1_0
        S_2 = S_2_0
        while time < m :
            u1 = U[2*i]
            u2 = U[2*i+1]
            i += 1
            tup = box_muller(u1, u2)
            if tup != None:
                zi, zj = tup
                count += 1
                S_1 = S_1 * np.exp((r-sigma_1**2/2)*delta_t + sigma_1*np.sqrt(delta_t)*zi)
                S_2 = S_2 * np.exp((r-sigma_2**2/2)*delta_t + sigma_2*np.sqrt(delta_t)*(rho*zi+np.sqrt(1-rho**2)*zj))
                time += 1
        
        sum_V += payoff_lockback_basket_call_option(S_1, S_2, K)
            
    avr_V = sum_V / n
        
    data = {
        'N_k': N_k,  
        'm=150': 150,  # fixed value
        'n': n, 
        '# of samples': count,
        'V^(n)': avr_V
    }
    df_result_page_6 = pd.concat([df_result_page_6, pd.DataFrame([data])], ignore_index=True)

In [23]:
df_result_page_6

,N_k,m=150,n,V^(n),# of samples
0,15000,150,50,12.714423,15000.000000
1,30000,150,100,12.041088,30000.000000
2,60000,150,200,10.775619,60000.000000
3,120000,150,400,11.064035,120000.000000
4,240000,150,800,11.082665,240000.000000
5,480000,150,1600,11.421017,480000.000000
6,960000,150,3200,11.187384,960000.000000
7,1920000,150,6400,11.324533,1920000.000000
8,3840000,150,12800,11.390086,3840000.000000
9,7680000,150,25600,11.393797,7680000.000000


**Comment on the convergence of the Monte Carlo pricer:** 

A larger number of paths benefits the MC in converging, and the converge benefit margin decreases in incresing the number of paths.

# Monte Carlo Simulation for the Heston Model

In [24]:
#The parameters for the asset
S0 = 50
sigma = 0.3
V0 = 0.09
lam = 4
V_mean = 0.35**2
eta = 0.25
q = 0
rho = -0.15

#The parameters for the put
T = 0.5
K = 50
r = 0.05

#The parameters for the model
m = 175
delta_t = T / m


In [25]:
p_true = bs_put(S0, K, T, r, q, sigma)
print(p_true)

3.5829339156412274


In [26]:
values = []
for k in range(6):
    n = 500 * (2**k)
    N = 2 * 175 * n # number of independent normal rvs needed
    U, _ = linear_congruential_generator(int(1.5*N)) # To generate N independent normal rvs, approximately needs 1.5*N independent uniform rvs
    sum_v = 0
    i = 0
    count = 0
    while count < 175 * n:
        time = 0
        S = S0
        V = V0
        while time < 175:
            u1 = U[2*i]
            u2 = U[2*i+1]
            i += 1
            tup = box_muller(u1,u2)
            if tup != None:
                zi,zj = tup
                count += 1
                S = S * np.exp((r - max(V,0)/2) * delta_t + np.sqrt(max(V,0) * delta_t) * zi)
                V = max(V,0) - lam * (max(V,0) - V_mean) * delta_t + eta * np.sqrt(max(V,0) * delta_t) * (rho * zi + np.sqrt(1 - rho**2) * zj)
                time += 1
        sum_v += put_payoff(S, K, T, r)
    value_hat = sum_v / n
    values.append(value_hat)

In [27]:
values

[3.9588521348855616,
 3.7352167699475767,
 3.961409951594242,
 4.025638237540043,
 4.014398561331073,
 3.9814912578023693]

In [28]:
#find the impied volatility
def bisection(f,x1,x2,tol):
    while max(abs(f(x1)),abs(f(x2)))>tol:
        mid = (x1+x2)/2
        if f(mid)*f(x1) < 0:
            x2 = mid
        else:
            x1 = mid
    return x1

In [29]:
imps = []
for v in values:
    def implied(sigma0):
        return bs_put(S0, K, T, r, q, sigma0) - v
    #print(implied(0.3))
    #print(implied(0.4))
    imp = bisection(implied,0.3,0.5,10**(-6))
    imps.append(imp)

In [30]:
imps

[0.3273270606994628,
 0.3110702037811279,
 0.3275129795074462,
 0.33218212127685537,
 0.3313650608062743,
 0.32897281646728516]